In [191]:
import torch
from sklearn.datasets import make_moons

In [192]:
X , y = make_moons(n_samples=1000, random_state=42)

In [193]:
X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

In [194]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [195]:
X_train[0], y_train[0]

(tensor([1.9469, 0.1784]), tensor(1.))

In [196]:
import torch
from torch import nn

In [197]:
device = "cuda"if torch.cuda.is_available() else "cpu"

In [198]:
class Moons(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1=nn.Linear(in_features=2, out_features=10)
    self.layer2=nn.Linear(in_features=10, out_features=10)
    self.layer3=nn.Linear(in_features=10, out_features=1)
    self.relu = nn.ReLU()

  def forward(self, x):
    return self.layer3(self.relu(self.layer2(self.relu(self.layer1(x)))))

In [199]:
model = Moons().to(device)
model

Moons(
  (layer1): Linear(in_features=2, out_features=10, bias=True)
  (layer2): Linear(in_features=10, out_features=10, bias=True)
  (layer3): Linear(in_features=10, out_features=1, bias=True)
  (relu): ReLU()
)

In [200]:
loss = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.1)

In [201]:
torch.sigmoid(model(X_train.to(device)[0:5]).squeeze())

tensor([0.5003, 0.5025, 0.5046, 0.5046, 0.5072], grad_fn=<SigmoidBackward0>)

In [202]:
y_train[0:5]

tensor([1., 0., 0., 0., 1.])

In [203]:
!pip install torchmetrics


In [204]:
from torchmetrics import Accuracy


In [205]:
accuracy = Accuracy(task='binary')

In [206]:
torch.manual_seed(42)

X_train, X_test = X_train.to(device), X_test.to(device)
y_train, y_test = y_train.to(device), y_test.to(device)

epcohs = 1000

for epoch in range(epcohs):

  model.train()

  y_logits = model(X_train).squeeze()
  y_pred = torch.sigmoid(y_logits)

  loss_train = loss(y_logits, y_train)
  accuracy_training = accuracy(y_pred, y_train)

  optimizer.zero_grad()

  loss_train.backward()

  optimizer.step()

  model.eval()
  with torch.no_grad():
    y_logits_test = model(X_test).squeeze()
    y_pred_test = torch.sigmoid(y_logits_test)

    loss_fn_test = loss(y_logits_test, y_test)
    accuracy_testing = accuracy(y_pred_test, y_test)

    if epoch % 20 == 0:
      print(f"{epoch}:training: acc:{accuracy_training} loss:{loss_train}, testing: acc:{accuracy_testing}, loss:{loss_fn_test}")


0:training: acc:0.3725000023841858 loss:0.6953642964363098, testing: acc:0.5, loss:0.6946237087249756
20:training: acc:0.7962499856948853 loss:0.6677951216697693, testing: acc:0.7699999809265137, loss:0.6691586375236511
40:training: acc:0.7637500166893005 loss:0.6258841156959534, testing: acc:0.7049999833106995, loss:0.6300035119056702
60:training: acc:0.7612500190734863 loss:0.5524411797523499, testing: acc:0.7200000286102295, loss:0.5611401200294495
80:training: acc:0.7900000214576721 loss:0.4605114758014679, testing: acc:0.7400000095367432, loss:0.47424113750457764
100:training: acc:0.8162500262260437 loss:0.3858400583267212, testing: acc:0.7649999856948853, loss:0.40273696184158325
120:training: acc:0.8412500023841858 loss:0.3363455832004547, testing: acc:0.7950000166893005, loss:0.35258370637893677
140:training: acc:0.8537499904632568 loss:0.3011782765388489, testing: acc:0.8399999737739563, loss:0.31434953212738037
160:training: acc:0.8662499785423279 loss:0.27399230003356934, te